### **Importing Libraries**

In [ ]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import *
from Game_and_Player import *
import time

### **Helper Functions**

In [ ]:

def simulate(game_state):
    """Simulates a random game from this node (play out the game to the end)."""
    game_state_new = copy.deepcopy(game_state)
    starting_player_index = game_state.current_player_index
    starting_player_name = game_state.players[starting_player_index].name

    while not game_state_new.is_terminal():
        legal_moves = game_state_new.get_legal_moves()
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        game_state_new.apply_move(move)

    rewards=game_state_new.tricks_won

    winner_name = max(rewards, key=rewards.get)


    winner_index = next(i for i, player in enumerate(game_state_new.players) if player.name == winner_name)


    return winner_index


In [ ]:
def monte_carlo_best_move(game_state, num_simulations=1000):
    legal_moves = game_state.get_legal_moves()
    move_wins = {(move.suit,move.rank): 0 for move in legal_moves}


    for move in legal_moves:
        for _ in range(num_simulations):
            sim_game_state = copy.deepcopy(game_state)
            sim_game_state.apply_move(move)

            winner = simulate(sim_game_state)

            if winner == game_state.current_player_index:
                move_wins[(move.suit,move.rank)] += 1
            # move_trials[move] += 1

    # Selecting the move with the highest win rate
    best_move = max(legal_moves, key=lambda m: (move_wins[(m.suit,m.rank)], m.rank))
    return best_move

In [ ]:
np.random.seed(24)
player_names = ["Alice", "Bob", "Charlie"]
roles = ["Teen", "Do", "Paanch"]
index = [0,1,2]

### **Monte Carlo vs. Random Play (Baseline)**

In [ ]:
def evaluate_monte_carlo(num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        print(f"Monte Carlo Player:{monte_carlo_player}")


        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:
                best_move = monte_carlo_best_move(game_state, num_simulations)
            else:
                legal_moves1 = game_state.get_legal_moves()

                if not legal_moves1:
                    print(f"No legal moves for player {game_state.current_player_index}. Ending game.")
                    break

                best_move = random.choice(legal_moves1)

            game_state.apply_move(best_move)



        rewards = game_state.tricks_won
        print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")



### **Monte Carlo Vs. Random Player**

In [ ]:
start=time.time()
evaluate_monte_carlo(num_games=500, num_simulations=100)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo Player:0
{'Alice': 3, 'Bob': 5, 'Charlie': 2}
Monte Carlo player did not win: Bob 
Monte Carlo Player:1
{'Alice': 2, 'Bob': 4, 'Charlie': 4}
Monte Carlo player won: Bob 
Monte Carlo Player:1
{'Alice': 2, 'Bob': 8, 'Charlie': 0}
Monte Carlo player won: Bob 
Monte Carlo Player:1
{'Alice': 1, 'Bob': 5, 'Charlie': 4}
Monte Carlo player won: Bob 
Monte Carlo Player:0
{'Alice': 5, 'Bob': 3, 'Charlie': 2}
Monte Carlo player won: Alice 
Monte Carlo Player:2
{'Alice': 6, 'Bob': 3, 'Charlie': 1}
Monte Carlo player did not win: Alice 
Monte Carlo Player:0
{'Alice': 6, 'Bob': 0, 'Charlie': 4}
Monte Carlo player won: Alice 
Monte Carlo Player:0
{'Alice': 4, 'Bob': 3, 'Charlie': 3}
Monte Carlo player won: Alice 
Monte Carlo Player:0
{'Alice': 5, 'Bob': 1, 'Charlie': 4}
Monte Carlo player won: Alice 
Monte Carlo Player:0
{'Alice': 6, 'Bob': 4, 'Charlie': 0}
Monte Carlo player won: Alice 
Monte Carlo Player:2
{'Alice': 1, 'Bob': 2, 'Charlie': 7}
Monte Carlo player won: Charlie 
Monte Carlo

### **Monte Carlo vs. Rule-Based Player**

In [ ]:
def rule_based_move(game_state):
    legal_moves = game_state.get_legal_moves()
    return max(legal_moves, key=lambda card: card.rank)


def evaluate_monte_carlo_rule_based(num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]



        game = Game(player_names, roles, index)
        game_state = GameState(game)

        game_state.current_player_index=random.choice(index)
        monte_carlo_player = random.choice(index)
        print(f"Monte Carlo Player:{monte_carlo_player}")


        while not game_state.is_terminal():

            if game_state.current_player_index == monte_carlo_player:
                best_move = monte_carlo_best_move(game_state, num_simulations)
            else:
                legal_moves1 = game_state.get_legal_moves()

                if not legal_moves1:
                    print(f"No legal moves for player {game_state.current_player_index}. Ending game.")
                    break

                best_move = rule_based_move(game_state)


            game_state.apply_move(best_move)
            # print(f"Applied move:{best_move}")
            # print(f"Table Cards:{game_state.table_cards}")




        rewards = game_state.tricks_won
        print(rewards)

        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        monte_carlo_name = player_names[monte_carlo_player]



        if winner_name == monte_carlo_name:
            monte_carlo_wins += 1
            win_results.append(1)
            print(f"Monte Carlo player won: {winner_name} ")
        else:
            win_results.append(0)
            print(f"Monte Carlo player did not win: {winner_name} ")

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Random Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")




In [ ]:
start=time.time()
evaluate_monte_carlo_rule_based(num_games=500, num_simulations=100)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Monte Carlo Player:2
{'Alice': 3, 'Bob': 2, 'Charlie': 5}
Monte Carlo player won: Charlie 
Monte Carlo Player:0
{'Alice': 4, 'Bob': 2, 'Charlie': 4}
Monte Carlo player won: Alice 
Monte Carlo Player:2
{'Alice': 1, 'Bob': 4, 'Charlie': 5}
Monte Carlo player won: Charlie 
Monte Carlo Player:2
{'Alice': 4, 'Bob': 4, 'Charlie': 2}
Monte Carlo player did not win: Alice 
Monte Carlo Player:0
{'Alice': 4, 'Bob': 2, 'Charlie': 4}
Monte Carlo player won: Alice 
Monte Carlo Player:0
{'Alice': 3, 'Bob': 4, 'Charlie': 3}
Monte Carlo player did not win: Bob 
Monte Carlo Player:0
{'Alice': 3, 'Bob': 2, 'Charlie': 5}
Monte Carlo player did not win: Charlie 
Monte Carlo Player:2
{'Alice': 3, 'Bob': 3, 'Charlie': 4}
Monte Carlo player won: Charlie 
Monte Carlo Player:0
{'Alice': 5, 'Bob': 3, 'Charlie': 2}
Monte Carlo player won: Alice 
Monte Carlo Player:1
{'Alice': 2, 'Bob': 4, 'Charlie': 4}
Monte Carlo player won: Bob 
Monte Carlo Player:1
{'Alice': 6, 'Bob': 2, 'Charlie': 2}
Monte Carlo player did n